In [56]:
from mip import *
import numpy as np
import time
from simulated_bifurcation import Ising
import json
from tqdm import tqdm

N_range = list(range(50,1001,50))
alpha_range = [1]

def generate_J_h(N_range,alpha_range):
    for N in N_range:
        for alpha in alpha_range:
            J = np.random.randint(-100,100,size=(N*alpha, N*alpha))/(10**6)
            J = (J + J.T)/2
            J = J - np.diag(np.diag(J))
            h = np.random.randint(-100,100,size=N*alpha)/(10**4)

            with open('test_gurobi/test_GB_J_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, J)

            with open('test_gurobi/test_GB_h_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, h)

            with open('test_gurobi/test_SB_J_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, J/4)

            with open('test_gurobi/test_SB_h_' + str(N) + '_' + str(alpha) + '.npy', 'wb') as f:
                np.save(f, h/2 - sum([J[:,i]/8 for i in range(N*alpha)]) - sum([J[i,:]/8 for i in range(N*alpha)]))

generate_J_h(N_range,alpha_range)

In [57]:
def create_optimize_gurobi(J,h,J_SB,h_SB,max_seconds=-1):
    time_1 = time.time()
    size = J.shape[0]

    model = Model("Ising", solver_name=GRB)
    model.verbose = 0
    u = [model.add_var(var_type=BINARY) for _ in range(size)]
    z = [[model.add_var(var_type=BINARY) for _ in range(size)] for _ in range(size)]

    for i in range(size):
        for j in range(size):
            model += z[i][j] <= u[i]
            model += z[i][j] <= u[j]
            model += z[i][j] >= u[i] + u[j] -1

    model.objective = minimize((-1/2)*quicksum(quicksum(J[i,j]*z[i][j] for i in range(size)) for j in range(size)) + quicksum(h[i]*u[i] for i in range(size)))
    if max_seconds == -1:
        model.optimize(max_seconds=300)
    else:
        model.optimize(max_seconds=max_seconds)

    spins = [1 if u[i].x >= 0.99 else -1 for i in range(size)]
    energy = (-1/2)*sum([sum([J_SB[i,j]*spins[j] for j in range(size)])*spins[i] for i in range(size)]) + sum([h_SB[i]*spins[i] for i in range(size)])[0]

    processing_time = time.time() - time_1
    return (processing_time,np.array(spins),energy)

In [58]:
def create_optimize_SB(J,h):
    time_1 = time.time()

    model = Ising(J,h)
    model.optimize(convergence_threshold = 35,
        sampling_period = 60,
        time_step = 0.01, 
        symplectic_parameter = 2,
        agents = 5,
        ballistic=False,
        heated=True,
        final_pressure=1.,
        pressure_slope=0.2)

    processing_time = time.time() - time_1
    return(processing_time, model.ground_state, model.energy)

In [59]:
def test_protocol(N_range, alpha_range):
    energy_precision = {}
    proc_times = {}
    energy_precision_stop = {}

    for N in tqdm(N_range):
        energy_precision[N] = {}
        proc_times[N] = {}
        energy_precision_stop[N] = {}
        for alpha in alpha_range:
            with open('test_gurobi/test_SB_J_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_SB_J = np.load(f)

            with open('test_gurobi/test_SB_h_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_SB_h = np.expand_dims(np.load(f),axis=1)

            with open('test_gurobi/test_GB_J_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_GB_J = np.load(f)

            with open('test_gurobi/test_GB_h_' + str(N) + '_' + str(alpha) + '.npy', 'rb') as f:
                test_GB_h = np.load(f)

            processing_time_sb, _, energy_sb = create_optimize_SB(test_SB_J,test_SB_h)
            processing_time_gb, _, energy_gb = create_optimize_gurobi(test_GB_J,test_GB_h,test_SB_J,test_SB_h)
            _, _, energy_gb_stopped = create_optimize_gurobi(test_GB_J,test_GB_h,test_SB_J,test_SB_h,processing_time_sb)

            energy_precision[N][alpha] = round((energy_sb - energy_gb)/abs(energy_gb),6)
            proc_times[N][alpha] = round(processing_time_gb/processing_time_sb,2)
            energy_precision_stop[N][alpha] = round((energy_gb_stopped - energy_sb)/abs(energy_sb),6)

    with open("test_gurobi/precisions_1.json", "w") as f:
        json.dump(energy_precision, f)
    
    with open("test_gurobi/times_1.json", "w") as f:
        json.dump(proc_times, f)

    with open("test_gurobi/precisions_early_stopping_1.json", "w") as f:
        json.dump(energy_precision_stop, f)

    return energy_precision,proc_times,energy_precision_stop

In [61]:
test_protocol(N_range,alpha_range)

  0%|          | 0/20 [00:00<?, ?it/s]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Bifurcated spins 0/5 Spins bifurcated in 0.187 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


  5%|▌         | 1/20 [00:00<00:14,  1.31it/s]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5 Spins bifurcated in 0.224 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 10%|█         | 2/20 [00:02<00:25,  1.43s/it]

...

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 4/5 Spins bifurcated in 1.01 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 45%|████▌     | 9/20 [01:50<03:51, 21.00s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 1/5⠸ Bifurcated spins 4/5 Spins bifurcated in 1.092 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


 50%|█████     | 10/20 [02:41<05:03, 30.37s/it]

✔ Building solver⠙ Building solver Solver built
✔ Spins evolving⠙ Spins evolving⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 0/5⠦ Bifurcated spins 0/5⠧ Bifurcated spins 0/5⠇ Bifurcated spins 0/5⠏ Bifurcated spins 0/5⠋ Bifurcated spins 0/5⠙ Bifurcated spins 0/5⠹ Bifurcated spins 0/5⠸ Bifurcated spins 0/5⠼ Bifurcated spins 0/5⠴ Bifurcated spins 4/5 Spins bifurcated in 1.232 sec.
✔ Retrieving ground state⠙ Retrieving ground state Ground state retrieved
Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16
